# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('/home/workspace/data/song_data/')

In [7]:
filepath = song_files[46]

In [8]:
df = pd.read_json(filepath,lines=True)

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
#song_data = list(df[['song_id', 'title', 'artist_id', 'year', 'duration']].values)
#song_data

song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data = song_data.values.tolist()
#song_data = song_data[0]
song_data[0]

['SOBZBAZ12A6D4F8742', 'Spanish Grease', 'AROUOZZ1187B9ABE51', 1997, 168.25424]

In [9]:
#song_table_insert="INSERT INTO songs VALUES(%s, %s, %s, %s, %s)"
song_table_insert

'\nINSERT INTO songs VALUES (%s, %s, %s, %s, %s)\n'

In [8]:
song_data

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
cur.execute(song_table_insert, song_data)
conn.commit()

IndexError: list index out of range

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [8]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data = artist_data.values.tolist()
artist_data = artist_data[0]
artist_data

['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [9]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [4]:
log_files = get_files('data/log_data')

In [5]:
filepath = log_files[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-29-events.json'

In [6]:
df = pd.read_json(filepath, lines=True)
df = df[df['page'] == 'NextSong']
df.head(1)
df[df.userId == ""]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [7]:
df = df[df['page'] == 'NextSong']
df = pd.to_datetime(df['ts'], unit='ms')
df.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [8]:
t = pd.Series(df)
t.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [9]:
list(t.dt.hour)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 4,
 4,
 4,
 5,
 8,
 8,
 10,
 11,
 11,
 11,
 11,
 12,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18

In [9]:
time_data = [list(t), list(t.dt.hour), list(t.dt.day), list(t.dt.week), list(t.dt.month), list(t.dt.year), list(t.dt.weekday)]
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [10]:
time_df = pd.DataFrame(time_data).transpose()
time_df.columns=column_labels
time_df

,start_time,hour,day,week,month,year,weekday
0,2018-11-29 00:00:57.796000,0,29,48,11,2018,3
1,2018-11-29 00:01:30.796000,0,29,48,11,2018,3
2,2018-11-29 00:04:01.796000,0,29,48,11,2018,3
3,2018-11-29 00:04:55.796000,0,29,48,11,2018,3
4,2018-11-29 00:07:13.796000,0,29,48,11,2018,3
5,2018-11-29 00:10:56.796000,0,29,48,11,2018,3
6,2018-11-29 00:11:04.796000,0,29,48,11,2018,3
7,2018-11-29 00:14:10.796000,0,29,48,11,2018,3
8,2018-11-29 00:16:02.796000,0,29,48,11,2018,3
9,2018-11-29 00:18:11.796000,0,29,48,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [11]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [9]:
df = df[df['page'] == 'NextSong']
user_df = df[['userId','firstName','lastName','gender','level']]
user_df
#user_df.to_csv('test')
#user_df[user_df['userId'].isna()]

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid
5,24,Layla,Griffin,F,paid
6,73,Jacob,Klein,M,paid
7,73,Jacob,Klein,M,paid
8,24,Layla,Griffin,F,paid
9,24,Layla,Griffin,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [10]:
for i, row in user_df.iterrows():
    print(i,row,user_table_insert)
    cur.execute(user_table_insert, row)
    conn.commit()

0 userId          73
firstName    Jacob
lastName     Klein
gender           M
level         paid
Name: 0, dtype: object 
INSERT INTO users VALUES (%s, %s, %s, %s, %s) ON CONFLICT(user_id) DO UPDATE SET level = EXCLUDED.level 

1 userId            24
firstName      Layla
lastName     Griffin
gender             F
level           paid
Name: 1, dtype: object 
INSERT INTO users VALUES (%s, %s, %s, %s, %s) ON CONFLICT(user_id) DO UPDATE SET level = EXCLUDED.level 

2 userId            24
firstName      Layla
lastName     Griffin
gender             F
level           paid
Name: 2, dtype: object 
INSERT INTO users VALUES (%s, %s, %s, %s, %s) ON CONFLICT(user_id) DO UPDATE SET level = EXCLUDED.level 

3 userId          73
firstName    Jacob
lastName     Klein
gender           M
level         paid
Name: 3, dtype: object 
INSERT INTO users VALUES (%s, %s, %s, %s, %s) ON CONFLICT(user_id) DO UPDATE SET level = EXCLUDED.level 

4 userId            24
firstName      Layla
lastName     Griffin
gender 

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [7]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [8]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [3]:
!python etl.py

73 files found in data/song_data
1/73 files processed.
2/73 files processed.
3/73 files processed.
4/73 files processed.
5/73 files processed.
6/73 files processed.
7/73 files processed.
8/73 files processed.
9/73 files processed.
10/73 files processed.
11/73 files processed.
12/73 files processed.
13/73 files processed.
14/73 files processed.
15/73 files processed.
16/73 files processed.
17/73 files processed.
18/73 files processed.
19/73 files processed.
20/73 files processed.
21/73 files processed.
22/73 files processed.
23/73 files processed.
24/73 files processed.
25/73 files processed.
26/73 files processed.
27/73 files processed.
28/73 files processed.
29/73 files processed.
30/73 files processed.
31/73 files processed.
32/73 files processed.
33/73 files processed.
34/73 files processed.
35/73 files processed.
36/73 files processed.
37/73 files processed.
38/73 files processed.
39/73 files processed.
40/73 files processed.
41/73 files processed.
42/73 files processed.
43/73 file